In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from tensorflow.keras.losses import SparseCategoricalCrossentropy

from art.attacks.evasion import ElasticNet
from art.estimators.classification import TensorFlowV2Classifier
from art.utils import load_dataset

import numpy as np
import pandas as pd

from sklearn import metrics

import joblib

2024-02-19 10:24:26.539332: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-19 10:24:26.571443: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-19 10:24:26.571475: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-19 10:24:26.572591: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-19 10:24:26.578254: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-19 10:24:26.578662: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [2]:
def calculate_performance_metrics(x_test, y_test, model):

    # Predictions
    y_pred = model.predict(x_test)
    y_pred_classes = np.argmax(y_pred, axis=1)
    
    report = metrics.classification_report(y_test, y_pred_classes, output_dict=True, zero_division=1)
    
    precision_macro = report['macro avg']['precision']
    precision_weighted = report['weighted avg']['precision']
    accuracy = report['accuracy']
    recall_macro = report['macro avg']['recall']
    recall_weighted = report['weighted avg']['recall']
    f1_macro = report['macro avg']['f1-score']
    f1_weighted = report['weighted avg']['f1-score']
    auc = metrics.roc_auc_score(y_test, y_pred, multi_class='ovr')

    print(f"Accuracy: {accuracy}\nAUC: {auc}")
    
    print("\nmacro")
    print(f"Precision: {precision_macro}\nRecall: {recall_macro}\nF1 Score: {f1_macro}")

    print("\nweighted")
    print(f"Precision: {precision_weighted}\nRecall: {recall_weighted}\nF1 Score: {f1_weighted}")
    print()
    
    # Confusion matrix for FNR, TNR, FPR, TPR
    cm = metrics.confusion_matrix(y_test, y_pred_classes)
    def calculate_rates(conf_matrix, class_index):
        tp = conf_matrix[class_index, class_index]
        fn = np.sum(conf_matrix[class_index, :]) - tp
        fp = np.sum(conf_matrix[:, class_index]) - tp
        tn = np.sum(conf_matrix) - (tp + fn + fp)
    
        fnr = fn / (fn + tp)
        tnr = tn / (tn + fp)
        fpr = fp / (fp + tn)
        tpr = tp / (tp + fn)
        return fnr, tnr, fpr, tpr

    # Calculate and aggregate rates
    fnrs, tnrs, fprs, tprs = [], [], [], []
    for i in range(cm.shape[0]):
        fnr, tnr, fpr, tpr = calculate_rates(cm, i)
        fnrs.append(fnr)
        tnrs.append(tnr)
        fprs.append(fpr)
        tprs.append(tpr)
    
    mean_fnr = np.mean(fnrs)
    mean_tnr = np.mean(tnrs)
    mean_fpr = np.mean(fprs)
    mean_tpr = np.mean(tprs)

    # Printing the mean metrics
    print(f"Mean FNR: {mean_fnr}\nMean TNR: {mean_tnr}\nMean FPR: {mean_fpr}\nMean TPR: {mean_tpr}")

In [3]:
x_train = pd.read_csv('/home/jovyan/WUSTL-IIoT/preprocessed_data/x_train.csv', low_memory=False)
y_train = pd.read_csv('/home/jovyan/WUSTL-IIoT/preprocessed_data/y_train.csv', low_memory=False)
x_val = pd.read_csv('/home/jovyan/WUSTL-IIoT/preprocessed_data/x_val.csv', low_memory=False)
y_val = pd.read_csv('/home/jovyan/WUSTL-IIoT/preprocessed_data/y_val.csv', low_memory=False)
x_test = pd.read_csv('/home/jovyan/WUSTL-IIoT/preprocessed_data/x_test.csv', low_memory=False)
y_test = pd.read_csv('/home/jovyan/WUSTL-IIoT/preprocessed_data/y_test.csv', low_memory=False)

In [4]:
from sklearn import preprocessing
s1 = preprocessing.MinMaxScaler()
s2 = preprocessing.MinMaxScaler()
s3 = preprocessing.MinMaxScaler()

x_train_scaled = s1.fit_transform(x_train)
x_val_scaled = s2.fit_transform(x_val)
x_test_scaled = s3.fit_transform(x_test)

x_train = np.copy(x_train_scaled)
x_val = np.copy(x_val_scaled)
x_test = np.copy(x_test_scaled)

In [5]:
model = joblib.load("/home/jovyan/WUSTL-IIoT/dl/dl_new/dnn.joblib")

2024-02-19 10:24:40.264789: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2256] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [11]:
np.unique(y_test)

array([0, 1, 2, 3, 4])

In [12]:
x_test_trimmed = np.concatenate((x_test[55500:55600], x_test[59200:59300]))
y_test_trimmed = np.concatenate((y_test[55500:55600], y_test[59200:59300]))

In [13]:
np.unique(y_test_trimmed)

array([0, 1, 2, 3, 4])

In [14]:
classifier = TensorFlowV2Classifier(
    model=model,
    nb_classes=5,
    input_shape=(48,),
    loss_object=tf.keras.losses.SparseCategoricalCrossentropy(),
)

In [15]:
# Initialize the Elastic Net attack
attack = ElasticNet(classifier=classifier, learning_rate=0.001)

# Generate adversarial test examples
x_test_adv = attack.generate(x=x_test_trimmed)

EAD:   0%|          | 0/200 [00:00<?, ?it/s]

EAD: 100%|██████████| 200/200 [46:24<00:00, 13.92s/it]


In [16]:
calculate_performance_metrics(x_test_adv, y_test_trimmed, model)

7/7 [==============================] - 0s 953us/step
Accuracy: 0.995
AUC: 1.0

macro
Precision: 0.9
Recall: 0.9875
F1 Score: 0.9268817204301076

weighted
Precision: 0.9975
Recall: 0.995
F1 Score: 0.995752688172043

Mean FNR: 0.0125
Mean TNR: 0.9989949748743718
Mean FPR: 0.0010050251256281408
Mean TPR: 0.9875


In [17]:
# Initialize the Elastic Net attack
attack = ElasticNet(classifier=classifier, learning_rate=0.01)

# Generate adversarial test examples
x_test_adv = attack.generate(x=x_test_trimmed)

EAD: 100%|██████████| 200/200 [46:33<00:00, 13.97s/it]


In [18]:
calculate_performance_metrics(x_test_adv, y_test_trimmed, model)

7/7 [==============================] - 0s 848us/step
Accuracy: 0.98
AUC: 1.0

macro
Precision: 0.75
Recall: 0.95
F1 Score: 0.7847619047619048

weighted
Precision: 0.99375
Recall: 0.98
F1 Score: 0.983904761904762

Mean FNR: 0.05
Mean TNR: 0.9959798994974876
Mean FPR: 0.004020100502512563
Mean TPR: 0.95


In [19]:
# Initialize the Elastic Net attack
attack = ElasticNet(classifier=classifier, learning_rate=0.1)

# Generate adversarial test examples
x_test_adv = attack.generate(x=x_test_trimmed)

EAD: 100%|██████████| 200/200 [46:48<00:00, 14.04s/it]


In [20]:
calculate_performance_metrics(x_test_adv, y_test_trimmed, model)

7/7 [==============================] - 0s 841us/step
Accuracy: 0.975
AUC: 0.9959798994974876

macro
Precision: 0.7166666666666666
Recall: 0.75
F1 Score: 0.5514285714285714

weighted
Precision: 0.9929166666666667
Recall: 0.975
F1 Score: 0.9780714285714285

Mean FNR: 0.25
Mean TNR: 0.9949748743718594
Mean FPR: 0.005025125628140703
Mean TPR: 0.75
